In [1]:
#Instalar o java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
#Baixar a versão mais recente do spark
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz


In [5]:
#Unzip o spark
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [6]:
#Criar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [7]:
#instalar a lib findspark que ajuda a localizar o Spark no sistema e importá-lo como uma biblioteca regular
!pip install -q findspark

In [8]:
#Importa a lib findspark
import findspark
findspark.init()

In [9]:
#criar spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('sparkcolab')\
        .getOrCreate()

In [12]:
#importar o dataset
from google.colab import files
arquivo = files.upload()

Saving movies.csv to movies.csv


In [13]:
#criar dataframe
df = spark.read.csv('movies.csv', header=True, inferSchema=True)

In [14]:
df.printSchema()

root
 |-- Film: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Lead Studio: string (nullable = true)
 |-- Audience score %: integer (nullable = true)
 |-- Profitability: double (nullable = true)
 |-- Rotten Tomatoes %: integer (nullable = true)
 |-- Worldwide Gross: string (nullable = true)
 |-- Year: integer (nullable = true)



In [15]:
df.show()

+--------------------+---------+--------------------+----------------+-------------+-----------------+---------------+----+
|                Film|    Genre|         Lead Studio|Audience score %|Profitability|Rotten Tomatoes %|Worldwide Gross|Year|
+--------------------+---------+--------------------+----------------+-------------+-----------------+---------------+----+
|Zack and Miri Mak...|  Romance|The Weinstein Com...|              70|  1.747541667|               64|        $41.94 |2008|
|     Youth in Revolt|   Comedy|The Weinstein Com...|              52|         1.09|               68|        $19.62 |2010|
|You Will Meet a T...|   Comedy|         Independent|              35|  1.211818182|               43|        $26.66 |2010|
|        When in Rome|   Comedy|              Disney|              44|          0.0|               15|        $43.04 |2010|
|What Happens in V...|   Comedy|                 Fox|              72|  6.267647029|               28|       $219.37 |2008|
| Water 

In [19]:
#selecionar colunas
df_movie = df.select('Film', 'Genre','Audience score %','Year')

In [20]:
df_movie.show()

+--------------------+---------+----------------+----+
|                Film|    Genre|Audience score %|Year|
+--------------------+---------+----------------+----+
|Zack and Miri Mak...|  Romance|              70|2008|
|     Youth in Revolt|   Comedy|              52|2010|
|You Will Meet a T...|   Comedy|              35|2010|
|        When in Rome|   Comedy|              44|2010|
|What Happens in V...|   Comedy|              72|2008|
| Water For Elephants|    Drama|              72|2011|
|              WALL-E|Animation|              89|2008|
|            Waitress|  Romance|              67|2007|
| Waiting For Forever|  Romance|              53|2011|
|     Valentine's Day|   Comedy|              54|2010|
|Tyler Perry's Why...|  Romance|              47|2007|
|Twilight: Breakin...|  Romance|              68|2011|
|            Twilight|  Romance|              82|2008|
|      The Ugly Truth|   Comedy|              68|2009|
|The Twilight Saga...|    Drama|              78|2009|
|The Time 

In [21]:
#filtro
df_movie.filter(df_movie['Year'] > 2009).show()

+--------------------+---------+----------------+----+
|                Film|    Genre|Audience score %|Year|
+--------------------+---------+----------------+----+
|     Youth in Revolt|   Comedy|              52|2010|
|You Will Meet a T...|   Comedy|              35|2010|
|        When in Rome|   Comedy|              44|2010|
| Water For Elephants|    Drama|              72|2011|
| Waiting For Forever|  Romance|              53|2011|
|     Valentine's Day|   Comedy|              54|2010|
|Twilight: Breakin...|  Romance|              68|2011|
|    The Back-up Plan|   Comedy|              47|2010|
|             Tangled|Animation|              88|2010|
|  Something Borrowed|  Romance|              48|2011|
|She's Out of My L...|   Comedy|              60|2010|
|Sex and the City Two|   Comedy|              49|2010|
|  Sex and the City 2|   Comedy|              49|2010|
|         Remember Me|    Drama|              70|2010|
|  Our Family Wedding|   Comedy|              49|2010|
|         

In [22]:
#pegar o max
from pyspark.sql.functions import max
df_max = df_movie.agg(max('Audience score %').alias('max_audience'))
df_max.show()

+------------+
|max_audience|
+------------+
|          89|
+------------+



In [23]:
df_max_audience = df_movie.filter(df_movie['Audience score %'] == 89)
df_max_audience.show()

+------------------+---------+----------------+----+
|              Film|    Genre|Audience score %|Year|
+------------------+---------+----------------+----+
|            WALL-E|Animation|              89|2008|
|A Dangerous Method|    Drama|              89|2011|
+------------------+---------+----------------+----+



In [25]:
#contar genero
from pyspark.sql.functions import count, col, asc, desc
df_sum = df_movie.groupBy('Genre').count()
df_sum.show(truncate=False)

+---------+-----+
|Genre    |count|
+---------+-----+
|Romance  |13   |
|Drama    |13   |
|Comdy    |1    |
|Romence  |1    |
|Fantasy  |1    |
|Animation|4    |
|Comedy   |41   |
|romance  |1    |
|Action   |1    |
|comedy   |1    |
+---------+-----+



In [26]:
#ordenação
df_sum.orderBy(col('count').desc()).show()

+---------+-----+
|    Genre|count|
+---------+-----+
|   Comedy|   41|
|    Drama|   13|
|  Romance|   13|
|Animation|    4|
|    Comdy|    1|
|  Romence|    1|
|   Action|    1|
|  romance|    1|
|  Fantasy|    1|
|   comedy|    1|
+---------+-----+



In [28]:
#utilizando o IN
df_movie.select('Film','Genre').filter(df_movie.Genre.isin('Drama','Comedy')).show()

+--------------------+------+
|                Film| Genre|
+--------------------+------+
|     Youth in Revolt|Comedy|
|You Will Meet a T...|Comedy|
|        When in Rome|Comedy|
|What Happens in V...|Comedy|
| Water For Elephants| Drama|
|     Valentine's Day|Comedy|
|      The Ugly Truth|Comedy|
|The Twilight Saga...| Drama|
|The Time Traveler...| Drama|
|        The Proposal|Comedy|
|The Invention of ...|Comedy|
|  The Heartbreak Kid|Comedy|
|         The Duchess| Drama|
|    The Back-up Plan|Comedy|
|She's Out of My L...|Comedy|
|Sex and the City Two|Comedy|
|  Sex and the City 2|Comedy|
|    Sex and the City|Comedy|
|         Remember Me| Drama|
|Rachel Getting Ma...| Drama|
+--------------------+------+
only showing top 20 rows



In [29]:
#utilizando o IN Negação
df_movie.select('Film','Genre').filter(~df_movie.Genre.isin('Drama','Comedy')).show()

+--------------------+---------+
|                Film|    Genre|
+--------------------+---------+
|Zack and Miri Mak...|  Romance|
|              WALL-E|Animation|
|            Waitress|  Romance|
| Waiting For Forever|  Romance|
|Tyler Perry's Why...|  Romance|
|Twilight: Breakin...|  Romance|
|            Twilight|  Romance|
|The Curious Case ...|  Fantasy|
|             Tangled|Animation|
|  Something Borrowed|  Romance|
|     P.S. I Love You|  Romance|
|             One Day|  Romance|
|      New Year's Eve|  Romance|
|    Music and Lyrics|  Romance|
|         Monte Carlo|  Romance|
|   Midnight in Paris|  Romence|
|       Made of Honor|    Comdy|
|             Killers|   Action|
|           Jane Eyre|  Romance|
|   Gnomeo and Juliet|Animation|
+--------------------+---------+
only showing top 20 rows



In [32]:
#Usando SQL
df_movie.createOrReplaceTempView('movies')
spark.sql("""
SELECT
    Genre,
    count(*) as qtd
FROM
    movies
GROUP BY
    Genre
ORDER BY
    qtd DESC
""").show(truncate=False)

+---------+---+
|Genre    |qtd|
+---------+---+
|Comedy   |41 |
|Romance  |13 |
|Drama    |13 |
|Animation|4  |
|Romence  |1  |
|Comdy    |1  |
|romance  |1  |
|comedy   |1  |
|Fantasy  |1  |
|Action   |1  |
+---------+---+

